In [1]:
# from transformers import BertTokenizer, TFBertForSequenceClassification
# from sklearn.model_selection import train_test_split
# import tensorflow as tf
# import pandas as pd

In [2]:
# # Load the data
# df = pd.read_csv('./data/balenced_data.csv')

In [3]:
# df.head()

In [4]:
# import pandas as pd

# # Assuming you have a DataFrame called df
# rows_with_nan = df[df.isnull().any(axis=1)]

# # Print the rows containing NaN values
# print(rows_with_nan)

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf

In [6]:
model_name = 'bert-base-uncased'
max_length = 128
batch_size = 32
learning_rate = 2e-5
number_of_epochs = 2

In [7]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Prepare the data
df = pd.read_csv('./data/balenced_data.csv')  # replace with your actual csv file

# drop nan values
df.dropna(axis=0, inplace=True)

sentences = df['text'].values
labels = df['Source'].values



In [9]:
# Split the data into train and test sets
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2)

In [10]:
# Function to convert lines of text into BERT input features
def convert_example_to_feature(review):
    return tokenizer.encode_plus(review, 
                                 add_special_tokens=True,  # add [CLS], [SEP]
                                 max_length=max_length,  # max length to truncate/pad
                                 pad_to_max_length=True,  # pad sentence to max length
                                 return_attention_mask=True,  # return attention mask
                                 )

In [11]:
# Convert train and test sentences into BERT input features
train_features = [convert_example_to_feature(sentence) for sentence in train_sentences]
test_features = [convert_example_to_feature(sentence) for sentence in test_sentences]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Felix\anaconda3\envs\bert\lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
# Function to map input features to tf.data.Dataset
def map_example_to_dict(input_ids, attention_masks, label):
    return {"input_ids": input_ids, "attention_mask": attention_masks}, label

In [13]:
def encode_examples(features, labels):
    input_ids_list = []
    attention_mask_list = []
    label_list = []
  
    for feature, label in zip(features, labels):
        input_ids_list.append(feature['input_ids'])
        attention_mask_list.append(feature['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, label_list)).map(map_example_to_dict)

In [16]:
# Prepare tf.data.Dataset for BERT
train_data = encode_examples(train_features, train_labels).shuffle(10000).batch(batch_size)
test_data = encode_examples(test_features, test_labels).batch(batch_size)

In [17]:
# Fine-tune the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [18]:
model.fit(train_data, epochs=number_of_epochs, validation_data=test_data)

Epoch 1/2


UnimplementedError: Graph execution error:

Detected at node 'Cast_1' defined at (most recent call last):
    File "c:\Users\Felix\anaconda3\envs\bert\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Felix\anaconda3\envs\bert\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\Felix\anaconda3\envs\bert\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Felix\anaconda3\envs\bert\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\Felix\anaconda3\envs\bert\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\Users\Felix\anaconda3\envs\bert\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\Felix\anaconda3\envs\bert\lib\asyncio\base_events.py"

In [4]:
NUM_CLASSES = df['Source'].nunique()

In [6]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUM_CLASSES)

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Tokenize the text data
inputs = tokenizer(list(df['text']), padding=True, truncation=True, max_length=512, return_tensors='pt')

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:

# Split the data into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(inputs['input_ids'], df['Source'], test_size=0.2)

In [ ]:
# Create data loaders
train_data = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

val_data = TensorDataset(val_inputs, val_labels)
val_loader = DataLoader(val_data, batch_size=16, shuffle=True)

In [ ]:
# Move the model to the GPU and define the loss and optimizer
model = model.to('cuda')
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
# Training loop
for epoch in range(NUM_EPOCHS):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, labels = batch
        input_ids = input_ids.to('cuda')
        labels = labels.to('cuda')
        outputs = model(input_ids, labels=labels)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()
